# Algorithm Comparison Notebook (5 Minute Time Limit)

This notebook tests all optimization algorithms with a **5-minute time budget** per algorithm.

**Comparisons:**
1. **Algorithms**: Hill Climbing, Simulated Annealing, GA Greedy, GA Tournament
2. **Heuristic**: With/Without heuristic initialization (GA only)
3. **Fitness Metrics**: L1 (Manhattan) vs L2 (SSD)
4. **Shapes**: Rect, Circle, Ellipse

All results tracked by **elapsed seconds** for fair comparison.

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Import from our modules
from genotype import Genotype, Rect, Circle, Ellipse
from utils import (
    load_img, initialize_grid, generate_phenotype,
    compute_fitness, compute_fitness_l1, get_jittered_grid_individual
)
from algorithms import (
    hill_climbing, simulated_annealing,
    ga_greedy, ga_tournament
)

In [ ]:
# Configuration
IMAGE_PATH = "./test.png"  # Change to your test image
MAX_TIME_MINUTES = 5       # 5 minutes per algorithm
MAX_TIME_SEC = MAX_TIME_MINUTES * 60  # Convert to seconds
MAX_ITER = 1000000         # High enough to not limit before time
POP_SIZE = 20              # Population size for GAs
GRID_SIZE = 5              # Grid size for local search init

print(f"Running each algorithm for {MAX_TIME_MINUTES} minutes ({MAX_TIME_SEC} seconds)")

# Load image
original_img, w, h = load_img(IMAGE_PATH)
print(f"Image size: {w}x{h}")

# Display original image
plt.figure(figsize=(6, 6))
plt.imshow(original_img)
plt.title("Original Image")
plt.axis("off")
plt.show()

## Helper Functions

In [ ]:
def run_experiment(name, algorithm_fn, **kwargs):
    """Run a single experiment and return results."""
    print(f"\n{'='*50}")
    print(f"Running: {name}")
    print(f"{'='*50}")
    
    start = time.time()
    best_geno, history = algorithm_fn(**kwargs)
    elapsed = time.time() - start
    
    final_fitness = history[-1][1] if history else float('inf')
    
    print(f"Final fitness: {final_fitness:,.0f}")
    print(f"Shapes: {len(best_geno.shapes)}")
    print(f"Time: {elapsed:.2f}s ({elapsed/60:.2f} min)")
    print(f"Iterations completed: {len(history)}")
    
    return {
        'name': name,
        'geno': best_geno,
        'history': history,
        'final_fitness': final_fitness,
        'time': elapsed,
        'shapes': len(best_geno.shapes),
        'iterations': len(history)
    }

def plot_fitness_by_time(results, title="Fitness by Time"):
    """Plot fitness evolution over elapsed time (seconds)."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Fitness over time
    ax1 = axes[0]
    for r in results:
        times = [h[0] for h in r['history']]
        fitness = [h[1] for h in r['history']]
        ax1.plot(times, fitness, label=r['name'], linewidth=1.5)
    ax1.set_xlabel('Time (seconds)')
    ax1.set_ylabel('Fitness (lower is better)')
    ax1.set_title(f'{title} - Fitness Evolution')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    
    # Final fitness bar chart
    ax2 = axes[1]
    names = [r['name'] for r in results]
    fitness_vals = [r['final_fitness'] for r in results]
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(names)))
    bars = ax2.bar(range(len(names)), fitness_vals, color=colors)
    ax2.set_xticks(range(len(names)))
    ax2.set_xticklabels(names, rotation=45, ha='right')
    ax2.set_ylabel('Final Fitness')
    ax2.set_title(f'{title} - Final Fitness After {MAX_TIME_MINUTES} min')
    
    # Add value labels on bars
    for bar, val in zip(bars, fitness_vals):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                 f'{val:,.0f}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()

def plot_convergence_rate(results, title="Convergence Rate"):
    """Show how quickly each algorithm improves."""
    fig, ax = plt.subplots(figsize=(10, 6))
    
    for r in results:
        if not r['history']:
            continue
        times = [h[0] for h in r['history']]
        fitness = [h[1] for h in r['history']]
        
        # Normalize to starting fitness for comparison
        start_fitness = fitness[0]
        improvement_pct = [(start_fitness - f) / start_fitness * 100 for f in fitness]
        
        ax.plot(times, improvement_pct, label=r['name'], linewidth=1.5)
    
    ax.set_xlabel('Time (seconds)')
    ax.set_ylabel('Improvement %')
    ax.set_title(f'{title} - Improvement Over Time')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def show_phenotypes(results, cols=4):
    """Display generated phenotypes."""
    n = len(results)
    rows = (n + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))
    axes = axes.flatten() if n > 1 else [axes]
    
    for i, r in enumerate(results):
        phenotype = generate_phenotype(r['geno'].w, r['geno'].h, r['geno'].shapes)
        axes[i].imshow(phenotype)
        axes[i].set_title(f"{r['name']}\nFitness: {r['final_fitness']:,.0f}\nShapes: {r['shapes']}")
        axes[i].axis('off')
    
    # Hide empty axes
    for i in range(n, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

---
## 1. All Algorithms Comparison (Rect, L2, No Heuristic)

Each algorithm runs for exactly 5 minutes using the `max_time` parameter.

In [ ]:
shape_class = Rect
fitness_fn = compute_fitness

results_basic = []

# Hill Climbing (5 min time limit)
start_geno = initialize_grid(original_img, w, h, shape_class, grid_size=GRID_SIZE)
results_basic.append(run_experiment(
    "Hill Climbing",
    hill_climbing,
    geno=start_geno, original_img=original_img, shape_class=shape_class,
    max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    track_by_time=True, fitness_fn=fitness_fn
))

# Simulated Annealing (5 min time limit)
start_geno = initialize_grid(original_img, w, h, shape_class, grid_size=GRID_SIZE)
results_basic.append(run_experiment(
    "Simulated Annealing",
    simulated_annealing,
    geno=start_geno, original_img=original_img, shape_class=shape_class,
    max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    track_by_time=True, fitness_fn=fitness_fn
))

# GA Greedy (5 min time limit)
results_basic.append(run_experiment(
    "GA Greedy",
    ga_greedy,
    original_img=original_img, shape_class=shape_class,
    pop_size=POP_SIZE, max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    track_by_time=True, fitness_fn=fitness_fn
))

# GA Tournament (5 min time limit)
results_basic.append(run_experiment(
    "GA Tournament",
    ga_tournament,
    original_img=original_img, shape_class=shape_class,
    pop_size=POP_SIZE, max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    track_by_time=True, fitness_fn=fitness_fn
))

In [ ]:
plot_fitness_by_time(results_basic, "All Algorithms (5 min each)")
plot_convergence_rate(results_basic, "All Algorithms")
show_phenotypes(results_basic)

---
## 2. Heuristic vs No Heuristic (GA only)

In [ ]:
results_heuristic = []

# GA Tournament WITHOUT heuristic
results_heuristic.append(run_experiment(
    "GA Tournament (No Heuristic)",
    ga_tournament,
    original_img=original_img, shape_class=Rect,
    pop_size=POP_SIZE, max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    fitness_fn=compute_fitness, use_heuristic=False, track_by_time=True
))

# GA Tournament WITH heuristic
results_heuristic.append(run_experiment(
    "GA Tournament (With Heuristic)",
    ga_tournament,
    original_img=original_img, shape_class=Rect,
    pop_size=POP_SIZE, max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    fitness_fn=compute_fitness, use_heuristic=True, track_by_time=True
))

# GA Greedy WITHOUT heuristic
results_heuristic.append(run_experiment(
    "GA Greedy (No Heuristic)",
    ga_greedy,
    original_img=original_img, shape_class=Rect,
    pop_size=POP_SIZE, max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    fitness_fn=compute_fitness, use_heuristic=False, track_by_time=True
))

# GA Greedy WITH heuristic
results_heuristic.append(run_experiment(
    "GA Greedy (With Heuristic)",
    ga_greedy,
    original_img=original_img, shape_class=Rect,
    pop_size=POP_SIZE, max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    fitness_fn=compute_fitness, use_heuristic=True, track_by_time=True
))

In [ ]:
plot_fitness_by_time(results_heuristic, "Heuristic Comparison")
plot_convergence_rate(results_heuristic, "Heuristic Comparison")
show_phenotypes(results_heuristic)

---
## 3. Fitness Metrics: L1 vs L2

In [ ]:
results_fitness = []

# Hill Climbing with L2
start_geno = initialize_grid(original_img, w, h, Rect, grid_size=GRID_SIZE)
results_fitness.append(run_experiment(
    "HC - L2 (SSD)",
    hill_climbing,
    geno=start_geno, original_img=original_img, shape_class=Rect,
    max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    fitness_fn=compute_fitness, track_by_time=True
))

# Hill Climbing with L1
start_geno = initialize_grid(original_img, w, h, Rect, grid_size=GRID_SIZE)
results_fitness.append(run_experiment(
    "HC - L1 (Manhattan)",
    hill_climbing,
    geno=start_geno, original_img=original_img, shape_class=Rect,
    max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    fitness_fn=compute_fitness_l1, track_by_time=True
))

# GA Tournament with L2
results_fitness.append(run_experiment(
    "GA Tournament - L2 (SSD)",
    ga_tournament,
    original_img=original_img, shape_class=Rect,
    pop_size=POP_SIZE, max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    fitness_fn=compute_fitness, track_by_time=True
))

# GA Tournament with L1
results_fitness.append(run_experiment(
    "GA Tournament - L1 (Manhattan)",
    ga_tournament,
    original_img=original_img, shape_class=Rect,
    pop_size=POP_SIZE, max_it=MAX_ITER, max_time=MAX_TIME_SEC,
    fitness_fn=compute_fitness_l1, track_by_time=True
))

In [ ]:
plot_fitness_by_time(results_fitness, "L1 vs L2 Fitness")
show_phenotypes(results_fitness)

---
## 4. Different Shapes: Rect vs Circle vs Ellipse

In [ ]:
results_shapes = []

for shape_name, shape_class in [("Rect", Rect), ("Circle", Circle), ("Ellipse", Ellipse)]:
    # Hill Climbing
    start_geno = initialize_grid(original_img, w, h, shape_class, grid_size=GRID_SIZE)
    results_shapes.append(run_experiment(
        f"HC - {shape_name}",
        hill_climbing,
        geno=start_geno, original_img=original_img, shape_class=shape_class,
        max_it=MAX_ITER, max_time=MAX_TIME_SEC,
        fitness_fn=compute_fitness, track_by_time=True
    ))
    
    # GA Tournament
    results_shapes.append(run_experiment(
        f"GA - {shape_name}",
        ga_tournament,
        original_img=original_img, shape_class=shape_class,
        pop_size=POP_SIZE, max_it=MAX_ITER, max_time=MAX_TIME_SEC,
        fitness_fn=compute_fitness, track_by_time=True
    ))

In [ ]:
plot_fitness_by_time(results_shapes, "Shape Comparison")
show_phenotypes(results_shapes, cols=3)

---
## 5. Summary Table

In [ ]:
import pandas as pd

# Combine all results
all_results = results_basic + results_heuristic + results_fitness + results_shapes

# Create summary dataframe
summary_df = pd.DataFrame([
    {
        'Experiment': r['name'],
        'Final Fitness': f"{r['final_fitness']:,.0f}",
        'Shapes': r['shapes'],
        'Iterations': r['iterations'],
        'Time (min)': f"{r['time']/60:.2f}"
    }
    for r in all_results
])

# Sort by final fitness
summary_df['_sort'] = [r['final_fitness'] for r in all_results]
summary_df = summary_df.sort_values('_sort').drop('_sort', axis=1).reset_index(drop=True)

print("\n" + "="*70)
print(f"SUMMARY - ALL EXPERIMENTS ({MAX_TIME_MINUTES} min each)")
print("="*70)
display(summary_df)

---
## 6. Best Result Visualization

In [ ]:
# Find best result
best_result = min(all_results, key=lambda r: r['final_fitness'])

print(f"🏆 Best Result: {best_result['name']}")
print(f"   Fitness: {best_result['final_fitness']:,.0f}")
print(f"   Shapes: {best_result['shapes']}")
print(f"   Iterations: {best_result['iterations']}")

# Display side by side
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(original_img)
axes[0].set_title("Original")
axes[0].axis("off")

best_phenotype = generate_phenotype(best_result['geno'].w, best_result['geno'].h, best_result['geno'].shapes)
axes[1].imshow(best_phenotype)
axes[1].set_title(f"Best: {best_result['name']}\nFitness: {best_result['final_fitness']:,.0f} | Shapes: {best_result['shapes']}")
axes[1].axis("off")

plt.tight_layout()
plt.show()

---
## 7. Fitness Improvement Analysis

In [ ]:
# Analyze improvement rate for basic algorithms
print("\nImprovement Analysis (Basic Algorithms):")
print("-" * 70)
print(f"{'Algorithm':<25} | {'Start Fitness':>15} | {'End Fitness':>15} | {'Improvement':>10}")
print("-" * 70)

for r in results_basic:
    if not r['history']:
        continue
    
    start_fit = r['history'][0][1]
    end_fit = r['history'][-1][1]
    improvement = (start_fit - end_fit) / start_fit * 100
    
    print(f"{r['name']:<25} | {start_fit:>15,.0f} | {end_fit:>15,.0f} | {improvement:>9.2f}%")